# Create table from csv files (basic)
Lee una serie de ficheros csv con los datos que se desean importar a nuevas tablas (una tabla para cada csv)<br>
__Uso__
1. Asignar valor a los parámetros 
1. Ejecutar la función tables_columns_2_rename

In [5]:
import csv
import glob
import os
from os.path import join


## Functions

In [10]:
def tables_columns_2_rename(drt: str, mask: str, to_screen: bool=False) -> [[str], [str]]:
    """
    Escribe el nombre de todas las tablas en drt/mask para ser renombradas como nombre de las tablas
    Extrae la cabecera de cada fichero csv y escribe en un fichero todas la cabeceras para que sean
    renombradas como las columnas de las tablas
    ----
    drt: directorio de los ficheros csv y de los ficheros de resultados
    mask: para seleccionar los ficheros a tratar, po ej *.csv
    ---
    return [name_tables, name_columns]
    name_tables: nombres de los ficheros
    name_columns: nombres de las columnas
    """
    files = glob.glob(join(drt, mask))
    name_tables = []
    name_columns = []

    for file in files:
        with open(file, mode='r', encoding='utf-8') as fi:
            rfi = csv.reader(fi, delimiter=',')
            table_name, _ = os.path.basename(file).split('.')
            name_tables.append(table_name)
            for ir, row in enumerate(rfi):
                if to_screen:
                    print(table_name)
                    print(row)
                    print('')
                for col1 in row:
                    if col1 not in name_columns:
                        name_columns.append(col1)
                break

    return name_tables, name_columns


def sets_get():
    """
    valores de los parámetros para escribir en el fichero sql donde se crean las tablas
    """
    sets = \
"""
-- PostgreSQL database dump

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;

"""    
    return sets
    
    
def create_table_sql(table_name, col_names, schema = ''):
    cols = [col1 + ' varchar' for col1 in col_names]
    cols = ',\n'.join(cols)
    if len(schema) > 0:
        schema = schema + '.'
    comment = f"comment on table {schema}{table_name} is 'CHJ Plan Hidrológico 21-27 {table_name}';\n"
    comment = comment.replace('_', ' ')
    
    comment_columns = [f"comment on columnn {schema}{table_name}.{col1} is '{col1}';" for col1 in col_names]
    comment_columns = '\n'.join(comment_columns)
    
    command = f"""
create table if not exists {schema}{table_name} (
{cols}    
);

{comment}
{comment_columns}
    """
    return command

## Set parameter values

In [33]:
# directorio y expression to get csv files
drt = r'E:\PPHH\chj_22_27\tablas'
mask = '*.csv'

# file_name_tables: nombre del fichero donde se grabarán el nombre d elas tablas
file_name_tables = join(drt, '01_table_names.txt')
# file_name_columns: nombre del fichero donde se grabarán el nombre de las columnas 
file_name_columns = join(drt, '02_colum_names.txt')

# fo_name: nombre del fichero donde se grabarán los comandos parara creación de las tablas
fo_name = join(drt, '03_create_tables.sql')

# schema: nombre del esquema de la DB donde se crearán las tablas
schema = 'tmp'


## Function 1. Write names of files and columns
Después de crearse el fichero deben darse los nuevos nobres manualmente en el fichero

In [34]:
tables, columns = tables_columns_2_rename(drt, mask)

with open(file_name_tables, 'w', encoding='utf-8') as fo:
    fo.write('name,new_name\n')
    for t1 in tables:
        fo.write(f'{t1},\n')

with open(file_name_columns, 'w', newline='', encoding='utf-8') as fo:
    fo.write('name,new_name\n')
    for c1 in columns:
        fo.write(f'{c1},\n')


In [12]:
tables

['anejo12_Tabla_121_Recurso_disponible',
 'anejo12_Tabla_122_bombeo_ie',
 'anejo12_Tabla_117_facies_hidroquimicas',
 'anejo12_Tabla_123_test1_balance',
 'anejo12_Tabla_124_test2_msf_cuanti',
 'anejo12_Tabla_127_test3_edas',
 'anejo12_Tabla_130_test4_intrusion_marina',
 'anejo12_Tabla_131_tests_mal_estado_gobal',
 'anejo12_Tabla_134_test_1_general_estado_quimico',
 'anejo12_Tabla_136_test_3_msf_quimico',
 'anejo12_Tabla_137_test 4_EDAS_quimico',
 'anejo12_Tabla_140_test 5_zonas_protegidas_quimico',
 'anejo12_Tabla_141_tests_mal_estado_quimico_representativo',
 'anejo12_Tabla_142_estado_global',
 'anejo12_Tabla_116_litologias']

In [ ]:
files = glob.glob(join(drt, mask))
with open(fo_name, 'w') as fo:
    fo.write(sets_get())
    for file in files:
        with open(file, mode='r', encoding='utf-8') as fi:
            rfi = csv.reader(fi, delimiter=',')
            table_name, _ = os.path.basename(file).split('.')
            for ir, row in enumerate(rfi):
                print(table_name)
                print(row)
                print('')
                sql_command = create_table_sql(table_name, row, schema = schema)
                break
            fo.write(f'{sql_command}\n\n')